In [130]:
import sys
sys.path.append('/Users/Jessica/Desktop/Coding/School/2802ICT/Assignment2')
from imports import *

In [131]:

class NN:

    def __init__(self):
        self.feature_train = []
        self.feature_test = []
        self.target_train = []
        self.target_test = []

    def arrange_data(self, train_file, test_file):
        
        train_set = self.open_data(train_file)
        test_set = self.open_data(test_file)

        self.target_train = train_set["label"]
        df = pd.DataFrame(train_set)
        self.feature_train = df.drop('label', axis=1)
        
        self.target_test = test_set["label"]
        df = pd.DataFrame(test_set)
        self.feature_test = df.drop('label', axis=1)
    
    def open_data(self, file):
        data_set = pd.read_csv(file)
        return data_set

    def sigmoid(z):
        return 1.0 / (1.0 + np.exp(-z))

    def driver(self):
        start = 0 

In [132]:

nn = NN()
nn.arrange_data("fashion-mnist_train.csv.gz", "fashion-mnist_test.csv.gz")


####################### USER SETUP AREA #######################
    
N_inp = 0
N_hidden = 0
N_out = 0    
    
####################### END OF USER SETUP #######################




      label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0         0       0       0       0       0       0       0       0       9   
1         1       0       0       0       0       0       0       0       0   
2         2       0       0       0       0       0       0      14      53   
3         2       0       0       0       0       0       0       0       0   
4         3       0       0       0       0       0       0       0       0   
...     ...     ...     ...     ...     ...     ...     ...     ...     ...   
9995      0       0       0       0       0       0       0       0       0   
9996      6       0       0       0       0       0       0       0       0   
9997      8       0       0       0       0       0       0       0       0   
9998      8       0       1       3       0       0       0       0       0   
9999      1       0       0       0       0       0       0       0     140   

      pixel9  ...  pixel775  pixel776  pixel777  pi